In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import glob
import torch
import seaborn as sns
from tqdm import tqdm

from sklearn.metrics import roc_auc_score, roc_curve, f1_score
from sklearn.preprocessing import label_binarize

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [59]:
df_drug = pd.read_csv('./data/entity_datasets/drug.csv')
df_drug_tr = df_drug[df_drug['is_test'] == False]
df_drug_te = df_drug[df_drug['is_test'] == True]
mapping_dict = dict(enumerate(df_drug_tr['condition'].astype('category').cat.categories))
mapping_dict_ = dict(enumerate(df_drug_te['condition'].astype('category').cat.categories))
df_drug_te.head()
mapping_dict == mapping_dict_

target = df_drug['cat'].values
is_test = df_drug.is_test.values
df_drug_te['cat'].value_counts()

,drugName,condition,review,rating,date,usefulCount,is_test,cat
107943,Mirtazapine,Depression,"""I&#039;ve tried a few antidepressants over th...",10.0,"February 28, 2012",22,True,10
107944,Bactrim,Urinary Tract Infection,"""Quick reduction of symptoms""",9.0,"September 29, 2017",3,True,26
107945,Contrave,Weight Loss,"""Contrave combines drugs that were used for al...",9.0,"March 5, 2017",35,True,28
107946,Cyclafem 1 / 35,Birth Control,"""I have been on this birth control for one cyc...",9.0,"October 22, 2015",4,True,6
107947,Copper,Birth Control,"""I&#039;ve had the copper coil for about 3 mon...",6.0,"June 6, 2016",1,True,6


True

6     9648
10    3095
23    2100
3     1908
2     1847
5     1380
28    1248
15    1231
20    1189
0     1126
12     827
27     811
11     808
14     783
25     660
1      648
7      639
17     583
29     579
4      573
9      525
16     524
8      485
24     469
18     454
26     431
21     398
22     387
19     387
13     381
Name: cat, dtype: int64

In [30]:
filename = list(glob.glob('./results/tuned/Llama-2-70b-hf/drug/condition/probes/*.sav'))[0]
layer_num = int(filename.split('/')[-1].split('_')[-1].split('.')[0])
act_path = f"./activation_datasets/Llama-2-70b-hf/drug/drug.last.condition.{layer_num}.pt"
acts = torch.load(act_path).dequantize()
test_activations = acts[is_test]
test_target = target[is_test]

model = pickle.load(open(filename, 'rb'))
test_pred = model.predict(test_activations)
test_prob = model.predict_proba(test_activations)

In [56]:
test_target[:10]
test_roc_auc = roc_auc_score(test_target, test_prob, multi_class='ovr', average=None)
predicted_labels = np.argmax(test_prob, axis=1)

# Compute F1 score for each class
f1_scores = f1_score(test_target, predicted_labels, average=None)
f1_scores.mean()

# Binarize the test_target to one-hot encoded form
test_target_bin = label_binarize(test_target, classes=np.arange(30))

fpr = dict()
tpr = dict()
roc_auc = dict()
n_classes = 30

for i in tqdm(range(n_classes)):
    fpr[i], tpr[i], _ = roc_curve(test_target_bin[:, i], test_prob[:, i])
    roc_auc[i] = roc_auc_score(test_target_bin[:, i], test_prob[:, i])

array([10, 26, 28,  6,  6, 18,  6, 27, 28, 15])

0.7471536635869889

100%|██████████| 30/30 [00:00<00:00, 108.46it/s]


In [53]:
test_roc_auc[10]

0.9690372258656917